In [1]:
import json

In [1]:
import json
import pandas as pd
from tqdm import tqdm
import requests

In [2]:
f = open('data_all/smart-2022-datasets/RL_relation_linking/wikidata/SMART2022-RL-wikidata-train.json')

data_add = json.load(f)

In [4]:
len(data_add)

24112

In [4]:
data_all = []

In [5]:
#SMART2022-RL-wikidata-train
for i in range(len(data_add)):
    text = data_add[i]['question']
    labels = data_add[i]['relation_labels']
    if len(labels) == len(set(labels)):
        cur_dict = {}
        cur_dict['labels'] = labels
        cur_dict['text'] = text
        data_all.append(cur_dict)

In [3]:
f = open('data_all/smart-2022-datasets/RL_relation_linking/wikidata/SMART2022-RL-wikidata-relation-vocabulary.json')
rels = json.load(f)

In [8]:
rels['P19']['label']

'place of birth'

### Training file creation

### WDSQ train

In [35]:
wdsq_train = pd.read_csv('data_all/train_ner_auto_wdsq.csv')

In [36]:
for i in tqdm(range(len(wdsq_train))):
    prop = wdsq_train.loc[i, 'property'] 
    if prop[0] == 'R':
        prop_upd = 'P'+prop[1:]
        wdsq_train.loc[i, 'prop_label'] = rels[prop_upd]['label'] + ' reversed'
    else:
        wdsq_train.loc[i, 'prop_label'] = rels[prop]['label'] 

100%|██████████| 19481/19481 [00:01<00:00, 13541.67it/s]


In [5]:
wdsq_train.head(n=1)

,subject,property,object,question,subject_text,flag,subject_text_add,subject_text_all,question_ner_auto,ner_auto
0,Q12439,R19,Q6106580,who is a musician born in detroit,"Detroit, 2014 Southeastern Michigan flood, Cit...",0.0,NaN,"Detroit, 2014 Southeastern Michigan flood, Cit...",who is a musician born in [START] detroit [END],detroit


In [37]:
for i in tqdm(range(len(wdsq_train))):
    text = wdsq_train.loc[i,'question']
    labels = wdsq_train.loc[i,'prop_label']
    cur_dict = {}
    cur_dict['labels'] = [labels]
    cur_dict['text'] = text
    data_all.append(cur_dict)

100%|██████████| 19481/19481 [00:00<00:00, 35876.86it/s]


In [13]:
data_all[-1]

{'labels': ['sex or gender'], 'text': 'What gender is gastón filgueira'}

### WDSQ validation

In [38]:
wdsq_val = pd.read_csv('data_all/val_ner_auto_wdsq.csv')

In [39]:
for i in tqdm(range(len(wdsq_val))):
    prop = wdsq_val.loc[i, 'property'] 
    if prop[0] == 'R':
        prop_upd = 'P'+prop[1:]
        wdsq_val.loc[i, 'prop_label'] = rels[prop_upd]['label'] + ' reversed'
    else:
        wdsq_val.loc[i, 'prop_label'] = rels[prop]['label']  

100%|██████████| 2821/2821 [00:00<00:00, 9260.72it/s]


In [40]:
for i in tqdm(range(len(wdsq_val))):
    text = wdsq_val.loc[i,'question']
    labels = wdsq_val.loc[i,'prop_label']
    cur_dict = {}
    cur_dict['labels'] = [labels]
    cur_dict['text'] = text
    data_all.append(cur_dict)

100%|██████████| 2821/2821 [00:00<00:00, 27096.15it/s]


In [22]:
data_all[-1]

{'labels': ['place of birth reversed'],
 'text': 'who is a person that was born in sao paulo'}

### RuBQ validation

In [41]:
rubq_dev = pd.read_csv('data_all/rubq_dev.csv')

In [42]:
rubq_dev = rubq_dev.query('tag == "1-hop"').reset_index(drop=True)

In [43]:
for i in tqdm(range(len(rubq_dev))):
    props_labs = []
    if pd.notna(rubq_dev.loc[i, 'props']):
        props = rubq_dev.loc[i, 'props'].split(', ')
        for elem in props:
            props_labs.append(elem.split(':')[-1])
    rubq_dev.loc[i, 'props_labs'] = ', '.join(props_labs)

100%|██████████| 373/373 [00:00<00:00, 4460.58it/s]


In [44]:
for i in tqdm(range(len(rubq_dev))):
    if pd.notna(rubq_dev.loc[i, 'props']):
        props = rubq_dev.loc[i, 'props_labs'].split(', ')
        props_labls = []
        for elem in props:
            if elem == 'altLabel':
                props_labls.append('altLabel')
            elif elem == 'label':
                props_labls.append('label')
            else:
                props_labls.append(rels[elem]['label'])
        rubq_dev.loc[i, 'prop_label'] = ', '.join(props_labls) 

100%|██████████| 373/373 [00:00<00:00, 4761.73it/s]


In [45]:
for i in tqdm(range(len(rubq_dev))):
    text = rubq_dev.loc[i,'question']
    labels = rubq_dev.loc[i,'prop_label']
    cur_dict = {}
    cur_dict['labels'] = [labels]
    cur_dict['text'] = text
    data_all.append(cur_dict)

100%|██████████| 373/373 [00:00<00:00, 19458.89it/s]


In [94]:
# rubq_test = pd.read_csv('data_all/rubq_test.csv')

In [4]:
mintaka_labeled = pd.read_csv('data_all_FALL/mintaka_test_dates.csv')

In [5]:
mintaka_labeled.head(n=1)

,Unnamed: 0,id,question,entities,category,complexityType,answerType,answerLabel,filt,one_hop_rels,subjects,possible_rel,ans,amount
0,0,3672da64,In what year did Kaley Cuoco play Cassie Bowde...,"[{'name': 'Q55635222', 'entityType': 'entity',...",movies,generic,date,2020,1.0,NaN,"Q55635222, Q16759",P582,2022-05-26T00:00:00Z,1.0


In [6]:
import requests

In [7]:
for i in tqdm(range(len(mintaka_labeled))):
    prop = mintaka_labeled.loc[i, 'prop'] 
    try:
        mintaka_labeled.loc[i, 'prop_label'] = rels[prop]['label']
    except:
        entity_id = prop
        r = requests.get("https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids={}&languages=en&formatversion=2&format=json".format(entity_id))
        ans = r.json()['entities'][entity_id]['labels']['en']['value']
        mintaka_labeled.loc[i, 'prop_label'] = ans

  0%|          | 0/57 [00:00<?, ?it/s]


KeyError: 'prop'

In [49]:
for i in tqdm(range(len(mintaka_labeled))):
    text = mintaka_labeled.loc[i,'question']
    labels = mintaka_labeled.loc[i,'prop_label']
    cur_dict = {}
    cur_dict['labels'] = [labels]
    cur_dict['text'] = text
    data_all.append(cur_dict)

100%|██████████| 380/380 [00:00<00:00, 18662.52it/s]


In [50]:
data_df = pd.DataFrame(columns=['question', 'labels'])
for i in tqdm(range(len(data_all))):
    if pd.notna(data_all[i]['labels'][0]):
        data_df.loc[len(data_df.index)] = [data_all[i]['text'], ', '.join(data_all[i]['labels'])]

100%|██████████| 46617/46617 [01:27<00:00, 531.75it/s]


In [ ]:
print(1)

In [51]:
data_df_filt = pd.DataFrame(columns=['question', 'labels'])
for elem in list(relations_dict.keys()):
    if relations_dict[elem] >= 2:
        cur_sub = data_df.query('labels == @elem').reset_index(drop=True)
        data_df_filt = data_df_filt.append(cur_sub, ignore_index=True)

/tmp/ipykernel_462/4143687005.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df_filt = data_df_filt.append(cur_sub, ignore_index=True)


In [52]:
data_all_filt = []
for i in tqdm(range(len(data_df_filt))):
    text = data_df_filt.loc[i,'question']
    labels = data_df_filt.loc[i,'labels']
    cur_dict = {}
    cur_dict['labels'] = [labels]
    cur_dict['text'] = text
    data_all_filt.append(cur_dict)

100%|██████████| 24588/24588 [00:00<00:00, 47124.81it/s]


In [54]:
json.dump(data_all_filt, open('data_all_FALL/training_data.json', 'w'))

In [23]:
f = open('data/training_data.json')

data = json.load(f)

In [24]:
data_df = pd.DataFrame(columns=['question', 'labels'])

In [25]:
relations_dict = dict()

In [26]:
for i in range(len(data_df)):
    elem = data_df.loc[i, 'labels']
    if elem not in relations_dict:
        relations_dict[elem] = 1
    else:
        relations_dict[elem] += 1

In [28]:
data_df_filt = pd.DataFrame(columns=['question', 'labels'])

In [29]:
for elem in list(relations_dict.keys()):
    if relations_dict[elem] >= 5:
        cur_sub = data_df.query('labels == @elem').reset_index(drop=True)
        data_df_filt = data_df_filt.append(cur_sub, ignore_index=True)
#         else:
#             cur_sub = data_df.query('labels == @elem').reset_index(drop=True).head(10)
#             data_df_filt = data_df_filt.append(cur_sub, ignore_index=True)

/tmp/ipykernel_10714/4252764261.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df_filt = data_df_filt.append(cur_sub, ignore_index=True)


In [30]:
data_all_filt = []
for i in tqdm(range(len(data_df_filt))):
    text = data_df_filt.loc[i,'question']
    labels = data_df_filt.loc[i,'labels']
    cur_dict = {}
    cur_dict['labels'] = [labels]
    cur_dict['text'] = text
    data_all_filt.append(cur_dict)

100%|██████████| 36464/36464 [00:00<00:00, 42390.43it/s]


In [33]:
json.dump(data_all_filt, open('data/training_data_filt.json', 'w'))

### Rubq_test_file creation

In [105]:
rubq_test = pd.read_csv('data_all/rubq_test.csv')

In [489]:
rubq_test.query('pred_label !=prop_label')

,uid,question,subject,props,query,tag,answers,props_labs,prop_label,pred_label
0,0,What can cause a tsunami?,Q8070,wdt:P828,SELECT ?answer \nWHERE {\n wd:Q8070 wdt:P828 ...,1-hop,"Q7944, Q60186, Q167903, Q2580904, Q5975740, Q7...",P828,has cause,"has cause, instance of"
5,6,What country owns the island of Tahiti?,Q42000,wdt:P17,SELECT ?answer \nWHERE {\n wd:Q42000 wdt:P17 ...,1-hop,Q142,P17,country,
6,8,Who created the Winter Palace in St. Petersburg?,Q182147,wdt:P84,SELECT ?answer \nWHERE {\n wd:Q182147 wdt:P84...,1-hop,Q299687,P84,architect,creator
9,11,What geographical feature is called Limpopo?,Q173017,wdt:P31,SELECT ?answer \nWHERE {\n wd:Q173017 wdt:P31...,1-hop,Q4022,P31,instance of,"instance of, located in/on physical feature"
10,12,What modern state occupies the territory where...,Q5680,wdt:P17,SELECT ?answer \nWHERE {\n wd:Q5680 wdt:P17 ?...,1-hop,Q796,P17,country,
...,...,...,...,...,...,...,...,...,...,...
1915,7195,What is the area of the sea of Azov?,Q35000,wdt:P2046,SELECT ?answer \nWHERE {\n wd:Q35000 wdt:P2046...,1-hop,37555,P2046,area,inception
1916,7196,What is the area of Belarus?,Q184,wdt:P2046,SELECT ?answer \nWHERE {\n wd:Q184 wdt:P2046 ?...,1-hop,207600.496983,P2046,area,inception
1917,7197,What is the area of ​​Kazakhstan?,Q232,wdt:P2046,SELECT ?answer \nWHERE {\n wd:Q232 wdt:P2046 ?...,1-hop,2724900,P2046,area,inception
1918,7198,When did Megalodon become extinct?,Q163873,wdt:P582,SELECT ?answer \nWHERE {\n wd:Q163873 wdt:P582...,1-hop,-3600000-01-01T00:00:00Z,P582,end time,inception


In [106]:
for i in tqdm(range(len(rubq_test))):
    props_labs = []
    if pd.notna(rubq_test.loc[i, 'props']):
        props = rubq_test.loc[i, 'props'].split(', ')
        for elem in props:
            props_labs.append(elem.split(':')[-1])
    rubq_test.loc[i, 'props_labs'] = ', '.join(props_labs)

100%|██████████| 2330/2330 [00:00<00:00, 8556.27it/s]


In [107]:
for i in tqdm(range(len(rubq_test))):
    if rubq_test.loc[i, 'tag'] == "0-hop":
         rubq_test.loc[i, 'prop_label'] = 'no-label'
    elif pd.notna(rubq_test.loc[i, 'props']):
        props = rubq_test.loc[i, 'props_labs'].split(', ')
        props_labls = []
        for elem in props:
            if elem == 'altLabel':
                props_labls.append('altLabel')
            elif elem == 'label':
                props_labls.append('label')
            else:
                try:
                    props_labls.append(rels[elem]['label'])
                except:
                    entity_id = elem
                    r = requests.get("https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids={}&languages=en&formatversion=2&format=json".format(entity_id))
                    ans = r.json()['entities'][entity_id]['labels']['en']['value']
                    props_labls.append(ans)
                    
        rubq_test.loc[i, 'prop_label'] = ', '.join(props_labls) 

100%|██████████| 2330/2330 [00:05<00:00, 419.27it/s]


In [54]:
# rubq_test = rubq_test.query('tag != "no-answer"').reset_index(drop=True)

In [49]:
test_data = []
for i in tqdm(range(len(rubq_test))):
    text = rubq_test.loc[i,'question']
    labels = rubq_test.loc[i,'prop_label']
    cur_dict = {}
    cur_dict['labels'] = [labels]
    cur_dict['text'] = text
    test_data.append(cur_dict)

100%|██████████| 1920/1920 [00:00<00:00, 22462.22it/s]


In [50]:
json.dump(test_data, open('data/test_data.json', 'w'))